In [2]:
import math
import copy

In [3]:
NONE = -1
ROBOT_ALLY = 0
ROBOT_ENEMY = 1
HOLE = 1
RADAR = 2
TRAP = 3
AMADEUSIUM = 4
ORE_UNKNOW = 5
PREDICT_AMADEUSIUM = 6
NEED_RADAR = 7
NEED_NO_HOLE = 8
NEED_REQUEST_RADAR = 9

In [4]:
class Pos:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __str__(self):
        return '({},{})'.format(self.x, self.y)
    
    def distance(self, pos):
        return abs(self.x - pos.x) + abs(self.y - pos.y)

class Entity(Pos):
    def __init__(self, x, y, type, id):
        super().__init__(x, y)
        self.type = type
        self.id = id


class Robot(Entity):
    def __init__(self, x, y, type, id, item):
        super().__init__(x, y, type, id)
        self.item = item

    def has_ore(self):
        return self.item == AMADEUSIUM

    def is_dead(self):
        return self.x == -1 and self.y == -1

    @staticmethod
    def move(x, y, message=""):
        print(f"MOVE {x} {y} {message}")

    @staticmethod
    def wait(message=""):
        print(f"WAIT {message}")

    @staticmethod
    def dig(x, y, message=""):
        print(f"DIG {x} {y} {message}")

    @staticmethod
    def request(requested_item, message=""):
        if requested_item == RADAR:
            print(f"REQUEST RADAR {message}")
        elif requested_item == TRAP:
            print(f"REQUEST TRAP {message}")
        else:
            raise Exception(f"Unknown item {requested_item}")

In [5]:
class Cell(Pos):
    def __init__(self, x, y, amadeusium, hole):
        super().__init__(x, y)
        self.amadeusium = amadeusium
        self.hole = hole

    def __str__(self):
        return 'x {} y {} amadeusium {} hole {}'.format(
            self.x,
            self.y,
            self.amadeusium,
            self.hole)
        
    def has_ore(self):
        if self.amadeusium == '?':
            return ORE_UNKNOW
        elif int(self.amadeusium) > 0 :
            return AMADEUSIUM
        else:
            return NONE

    def has_hole(self):
        #print("has_hole i {} j {} amadeusium {} hole {}".format(
        #    self.x,
        #    self.y,
        #    self.amadeusium,
        #    self.hole), file=sys.stderr)
        return self.hole

    def update(self, amadeusium, hole):
        self.amadeusium = amadeusium
        self.hole = hole

In [6]:
class CellInfo:
    def __init__(self):
        self.next_amadeusium = -1
        self.next_hole = 0
        self.my_trap = 0
        self.my_radar = 0
        self.has_opponent = 0
        self.opponent_trap = 0
        self.opponent_radar = 0
        
        
    def __str__(self):
        return '(ore {},hole {},trap {}, radar {}, opponent {})'.format(
            self.next_amadeusium,
            self.next_hole,
            self.trap,
            self.radar,
            self.has_opponent)
        
    def predict(self, previous):
        self = copy.copy(previous)        
        
    def correction(self, cell):
        if cell.amadeusium != '?' :
            self.next_amadeusium = self.amadeusium
        #if self.next_hole != 
            

In [7]:
class RobotInfo:
    def __init__(self):
        self.next_x = 0
        self.next_y = 0
        self.has_dig = 0
        self.has_ore = 0
        self.has_trap = 0
        self.has_radar = 0
        
    def __str__(self):
        return '(next_x {},next_y {},has_dig {}, has_ore {}, has_trap {} has_radar {})'.format(
            self.next_x,
            self.next_y,
            self.has_dig,
            self.has_ore,
            self.has_trap,
            self.has_radar)
    
    def predict(self,previous):
        self = copy(previous)
        
    def correction(self, robot):
        self.next_x = robot.x
        self.next_y = robot.y
        if robot.item == AMADEUSIUM:
            self.has_ore = 1
        elif robot.item == TRAP:
            self.has_trap = 1
        elif robot.item == RADAR:
            self.has_radar = 1
    

In [8]:
class Grid:
    def __init__(self):
        self.cell = []
        for y in range(height):
            for x in range(width):
                self.cell.append(Cell(x, y, 0, 0))
        self.cell_info = []
        for y in range(height):
            for x in range(width):
                self.cell_info.append(CellInfo())
        self.index = 0
        self.size = height * width
        self.find = NONE

    def get_cell(self, x, y):
        if width > x >= 0 and height > y >= 0:
            return self.cell[x + width * y]
        return None
    
    def get_cell_info(self, x, y):
        if width > x >= 0 and height > y >= 0:
            return self.cell_info[x + width * y]
        return None

    def get_index(self, x, y):
        if width > x >= 0 and height > y >= 0:
            return x + width * y
        return -1
        
    
    def get_index_cross(self,x,y):
        # Back
        t_tuple = [-1,-1,-1,-1]
        if width > x - 1 >= 0 and height > y >= 0:
            t_tuple[3] = x - 1 + width * y
        # Front
        if width > x + 1 >= 0 and height > y >= 0:
            t_tuple[0] = x - 1 + width * y
        # Up
        if width > x >= 0 and height > y - 1 >= 0:
            t_tuple[1] = x + width * (y - 1)
        # Down
        if width > x >= 0 and height > y + 1 >= 0:
            t_tuple[2] = x + width * (y + 1)
        t_tuple = tuple(t_tuple)
        return t_tuple
    
    def get_index_square(self,x,y):
        t_true = True
        if width <= x + 1 or x - 1 < 1 or height <= y + 1 or (y - 1) < 0:
            t_true = False
        if t_true == False:
            return None
        t_tuple = ((x-1,y-1),(x,y-1),(x+1,y-1),(x-1,y),(x,y),(x+1,y),(x-1,y+1),(x,y+1),(x+1,y+1))
        #print('tuple {}'.format(t_tuple))
        t_result = []
        for i in t_tuple:
            (x,y) = i
            t_result.append(x + width * y)
        t_tuple = tuple(t_result)
        
        return t_tuple

    def __getitem__(self,index):
        if index < 0 or index >= height * wight:
            return None
        return (self.cell[index], self.cell_info[index])
    
    def __iter__(self):
        self.index = 0
        return self

    def __next__(self):
        while True:
            if self.index == self.size:
                raise StopIteration
        
            item_cell = self.cell[self.index]
            item_cell_info = self.cell_info[self.index]
            self.index = self.index + 1
        
            #print('search i {}'.format(self.index))
        
            if self.find == AMADEUSIUM and item_cell.has_ore() > 0 :
                break
                
            if self.find == PREDICT_AMADEUSIUM and item_cell_info.next_amadeusium > 0 :
                break
                
            if self.find == NEED_RADAR :
                x = self.index % width
                y = math.trunc((self.index - x) / width)
                t_tuple = self.get_index_square(x,y)
                if t_tuple is None:
                    continue
                    
                has_found = True
                #print('new tuple')
                for i in t_tuple:
                    #print('i {} has_ore {}'.format(i,self.cell[i].has_ore()))
                    if self.cell[i].has_ore() != -1:
                        has_found = False
                if has_found == True :
                    break
            
            if self.find == NEED_NO_HOLE and item_cell.has_hole() != HOLE :
                break
            
        return (item_cell, item_cell_info)


In [9]:
class GridState:
    def __init__(self):
        self.grid = []
        self.index = 0
        self.size = 0
    
    def append(self, Grid):
        self.grid.append(Grid)
        self.size = self.size + 1
    
    def previous(self):
        return self[-2]
    
    def current(self):
        return self[-1]
    
    def __iter__(self):
        self.index = 0
        return self

    def __next__(self):
        if self.index == self.size:
            raise StopIteration
        item_grid = self.grid[self.index]
        self.index = self.index + 1
        return item_grid

    def __getitem__(self,index):
        item_grid = self.grid[self.index]
        return item_grid


In [10]:
class RobotState:
    def __init__(self):
        self.robot = []
        self.robot_state = []
        self.index = 0
        self.size = 0
    
    def append(self, Robot, RobotInfo):
        self.robot.append(Robot)
        self.robot_state.append(RobotState)
        self.size = self.size + 1
    
    def previous(self):
        return self[-2]
    
    def current(self):
        return self[-1]
    
    def __iter__(self):
        self.index = 0
        return self

    def __next__(self):
        if self.index == self.size:
            raise StopIteration
        (robot, robot_state) = (self.robot[self.index], self.robot_state[self.index])
        self.index = self.index + 1
        return (robot,robot_state)

    def __getitem__(self,index):
        (robot, robot_state) = (self.robot[self.index], self.robot_state[self.index])
        return (robot,robot_state)


In [11]:
class Strategy():
    def __init__(self, GridState, RobotState):
        self.index = 0
        self.size = width * height
        self.grid_state = GridState
        self.robot_state = RobotState
        self.find_ore = False
        self.find_surely_ore = False
        self.find_correct_place_radar = False
        
        self.previous_grid = self.grid_state[-2]
        self.current_grid = self.grid_state[-1]
        self.previous_agent = self.robot_state[-2]
        self.current_agent = self.robot_state[-1]
        
        self.radar_cooldown = 100
        self.trap_cooldown = 100
        
        current_grid_iter = iter(self.current_grid)
        for (cell,cell_info) in self.previous_grid:
            (current_cell, current_cell_info) = next(current_grid_iter)
            current_cell_info.predict(cell_info)
            current_cell_info.correction(current_cell)
        
        
    def find_amadeusium(self):
        grid.find = AMADEUSIUM
        min_distance = 100
        distance = 0
        keep = None
        (robot,robot_info) = self.current_agent
        for (cell,cell_info) in self.current_grid:
            distance = robot.distance(cell)
            if distance < min_distance:
                min_distance = distance
                keep = (min_distance,cell,cell_info)
        return keep
    
    def find_amadeusium_from_predict(self):
        grid.find = PREDICT_AMADEUSIUM
        min_distance = 100
        distance = 0
        keep = None
        (robot,robot_info) = self.current_agent
        for (cell,cell_info) in self.current_grid:
            distance = robot.distance(cell)
            if distance < min_distance:
                min_distance = distance
                keep = (min_distance,cell,cell_info)
        return keep

    def find_radar_correct_place(self):
        grid.find = NEED_RADAR
        min_distance = 100
        distance = 0
        keep = None
        (robot,robot_info) = self.current_agent
        for (cell,cell_info) in self.current_grid:
            distance = robot.distance(cell)
            if distance < min_distance:
                min_distance = distance
                keep = (min_distance,cell,cell_info)
        return keep

    def find_no_hole(self):
        gred.find = NEED_NO_HOLE
        min_distance = 100
        distance = 0
        keep = None
        (robot,robot_info) = self.current_agent
        for (cell,cell_info) in self.current_grid:
            distance = robot.distance(cell)
            if distance < min_distance:
                min_distance = distance
                keep = (min_distance,cell,cell_info)
        return keep
    
    

In [12]:
width = 30
height = 15

In [13]:
grid_state = GridState()
robot_state = RobotState()


In [14]:
grid = Grid()
robot = Robot(3,6,0,0,-1)
robot_info = RobotInfo()

In [15]:
grid_state.append(grid)

In [16]:
robot_state.append(robot,robot_info)

In [17]:
robot_state.size

1

In [18]:
grid_state.append(grid)

In [19]:
robot_state.append(robot, robot_info)

In [27]:
strategy = Strategy(grid_state, robot_state)

AttributeError: 'CellInfo' object has no attribute 'amadeusium'

In [21]:
result = strategy.find_amadeusium()
result

In [22]:
type(result)

NoneType

In [23]:
result = strategy.find_amadeusium_from_predict()
result


In [24]:
type(result)

NoneType

In [25]:
result = strategy.find_radar_correct_place()
result

(0, <__main__.Cell at 0x7f9b6042d908>, <__main__.CellInfo at 0x7f9b60458cf8>)

In [26]:
#type(result)
t_cell, t_cell_info = result
print(t_cell)
print(result)

ValueError: too many values to unpack (expected 2)

In [ ]:
(agent, agent_info) = robot_state.current()
print(agent)